❗️ **If there is a problem rendering this article (garbled or incorrect format), please feel free to visit my Github repository for this course:**

https://github.com/jimcui3/Introduction-to-Machine-Learning

# *Introduction to Machine Learning* Algorithms and Realizations 4

## By Jiaheng Cui
>In this chapter, we'll talk about Decision Trees, a widely-used model in supervised learning. We'll introduce three basic algorithms to generate a decision tree: ID3, C4.5, CART. Meanwhile, we'll introduce prunning techniques.

## 1.Decision Trees
### (1) Introduction

### (2) Feature selection


## 2.ID3 Algorithm
### (1) Entropy and Information gain

### (2) ID3 Algorithm
Input: training instances $X = \left[\begin{matrix}  x_{11} & \cdots & x_{1p}\\ \vdots & \ddots & \vdots \\ x_{n1} & \cdots & x_{np} \\ \end{matrix}\right]$, label vector $y = (y_1, y_2, ..., y_n)^T$.

Output: parameter vector $\hat{\beta}$

Step1: load $X$ and $y$, note that there isn't a "1" in the front of each row, we'll add them manually. 

Step2: let $\hat{\beta} = (X^T X)^{-1}X^T y$, so the model we trained is $y = X \hat{\beta}$.

### (3) Code

In [1]:
import numpy as np
import pandas as pd
from numpy import log2 as log # "log" is the logarithm to base 2

In [2]:
class ID3_Classification_Tree():
    def __init__(self, df):
        self.df = df
        self.label = list(df)
     
    
    # Calculate the entropy of different attributes
    def find_entropy(self, df):
        entropy = 0
        Class = df.keys()[-1] # self.df.keys()[-1] is the last column, i.e. the label
        values = df[Class].unique()# We want to devide df by the label, so we find the distinct labels
        
        for value in values:
            fraction = df[Class].value_counts()[value]/len(df[Class])# fraction_i = |C_k|/|D|
            
            if(fraction != 0): # Let 0*log0 = 0, otherwise we cannot compute the entropy
                entropy += -fraction * log(fraction)# entropy_i = -|C_k|/|D|* log(|C_k|/|D|)
                
        return entropy

    
    # Calculate the entropy within an attribute (i.e. subsets)
    def find_entropy_attribute(self, df, attribute):
        Class = df.keys()[-1]
        target_variables = df[Class].unique()
        variables = df[attribute].unique() # Find the distinct features within the attribute
        conditional_entropy = 0
        
        for variable in variables:
            entropy = 0
            
            for target_variable in target_variables:
                num = len(df[attribute][df[attribute] == variable][df[Class] == target_variable]) # |D_ij|
                den = len(df[attribute][df[attribute] == variable]) # |D_i|
                fraction = num/den # |D_ij|/|D_i|
                
                if(fraction != 0):
                    entropy += -fraction * log(fraction) # entropy is the entropy of one of the values in the attribute
                    
            fraction2 = den/len(df) # |D_i|/|D|
            conditional_entropy += fraction2 * entropy # conditional_entropy is the conditional entropy under the attribute
            
        return conditional_entropy
    
    
    # Find the attribute that maximizes the information gain
    def find_winner(self, df): 
        competitors = [] # competitors is all possible moves of an node
        
        for key in df.keys()[:-1]: # df.keys()[:-1] is all the attributes of df except for the label
            competitors.append(self.find_entropy(df) - self.find_entropy_attribute(df, key)) # Compute the information gain of key
            
        return df.keys()[:-1][np.argmax(competitors)] 

    
    # Get the subset of a node by its value
    def get_subtable(self, df, node, value):
        return df[df[node] == value].reset_index(drop = True)

    
    def buildTree(self, df): 
        Class = df.keys()[-1]
        node = self.find_winner(df) # Find the attribute that maximizes the information gain
        attValue = np.unique(df[node]) # Get distinct value of the attribute
        tree = {}
        tree[node] = {}

        for value in attValue:
            subtable = self.get_subtable(df, node,value)
            clValue, counts = np.unique(subtable[subtable.keys()[-1]], return_counts = True)

            if (len(counts) == 1):# Subset is pure, stop generating the tree from this path
                tree[node][value] = clValue[0]
                
            else:        
                tree[node][value] = self.buildTree(subtable) # Subset is not pure, so call the function recursively
                
        return tree
    
    
    def train(self):
        self.tree = self.buildTree(self.df)
    
    
    def predict_iteration(self, tree_dict, test_column):
        firstStr = list(tree_dict.keys())[0] # Find the attribute of the first node
        secondDict = tree_dict[firstStr] # Find the subtree of this node
        featureIndex = self.label.index(firstStr) # Find the subset of that subtree

        for key in secondDict.keys():
            if(test_column[featureIndex] == key):
                if(isinstance(secondDict[key], dict)): # If this node is a dict, this node is a inner node, so call the function recursively.
                    classlabel = self.predict_iteration(secondDict[key], test_column)

                else: # This node is a leaf, just output the result.
                    classlabel = secondDict[key]
                    
        return classlabel
    
    
    def predict(self, tree_dict, test_data):
        predicted_labels = list()
        
        for i in range(test_data.shape[0]):
            predicted_labels.append(self.predict_iteration(tree_dict, test_data[i,:])) # Evaluate each line.
            
        return predicted_labels

Below is the dataset which we'll use to train the model. We put it into a dataframe named `df`.

In [3]:
training_set = np.array([['较高','是','是','否','感冒'],
                         ['非常高','否','否','否','不感冒'],
                         ['非常高','是','否','是','感冒'],
                         ['正常','是','是','是','感冒'],
                         ['正常','否','否','是','不感冒'],
                         ['较高','是','否','否','感冒'],
                         ['较高','是','否','是','感冒'],
                         ['非常高','是','是','否','感冒'],
                         ['较高','否','是','是','感冒'],
                         ['正常','是','否','否','不感冒'],
                         ['正常','是','否','是','感冒'],
                         ['正常','否','是','是','感冒'],
                         ['较高','否','否','否','不感冒'],
                         ['非常高','否','是','否','感冒'],
                         ['非常高','否','是','否','感冒'],
                         ['较高','否','否','是','感冒']])

labels = np.array(['体温','流鼻涕','肌肉疼','头疼','感冒'])

df = pd.DataFrame(training_set, columns = labels)

Now we train an ID3 classification tree model `decision_tree`:

In [4]:
decision_tree = ID3_Classification_Tree(df)
decision_tree.train()

We can see the decision tree by calling `print(decision_tree.tree)`, the result will be a dict:

In [5]:
t = decision_tree.tree
print(t)

{'肌肉疼': {'否': {'流鼻涕': {'否': {'体温': {'正常': '不感冒', '较高': {'头疼': {'否': '不感冒', '是': '感冒'}}, '非常高': '不感冒'}}, '是': {'体温': {'正常': {'头疼': {'否': '不感冒', '是': '感冒'}}, '较高': '感冒', '非常高': '感冒'}}}}, '是': '感冒'}}


Note: we can use the python package **graphviz** to visualize a decision tree and output a pdf file. Those who are interested can find it here:

https://github.com/xflr6/graphviz

We can predict new data by calling `decision_tree.predict(t, test_set)`, where t is the tree, and test_set is the new dataset:

In [6]:
test_set = np.array([['较高','否','否','否'], ['非常高','否','是','否']])
decision_tree.predict(t, test_set)

['不感冒', '感冒']

## 3.C4.5 Algorithm
### (1) Information gain ratio

### (2) Basic thoughts of C4.5 algorithm

### (3) PEP Post-prunning

### (4) Algorithm
Input: training instances $X = \left[\begin{matrix}  x_{11} & \cdots & x_{1p}\\ \vdots & \ddots & \vdots \\ x_{n1} & \cdots & x_{np} \\ \end{matrix}\right]$, label vector $y = (y_1, y_2, ..., y_n)^T$.

Output: parameter vector $\hat{\beta}$

Step1: load $X$ and $y$, note that there isn't a "1" in the front of each row, we'll add them manually. 

Step2: let $\hat{\beta} = (X^T X)^{-1}X^T y$, so the model we trained is $y = X \hat{\beta}$.

### (5) Code

## 4.CART Algorithm
### (1) Gini index

### (2) CART classification tree

### (3) CART regression tree

### (4) CCP Post-prunning

### (5) Algorithm
Input: training instances $X = \left[\begin{matrix}  x_{11} & \cdots & x_{1p}\\ \vdots & \ddots & \vdots \\ x_{n1} & \cdots & x_{np} \\ \end{matrix}\right]$, label vector $y = (y_1, y_2, ..., y_n)^T$.

Output: parameter vector $\hat{\beta}$

Step1: load $X$ and $y$, note that there isn't a "1" in the front of each row, we'll add them manually. 

Step2: let $\hat{\beta} = (X^T X)^{-1}X^T y$, so the model we trained is $y = X \hat{\beta}$.

### (6) Code

If you want to use decision tree algorithms from **sklearn**, please note that they only accept numerical values, but not categorical values. So you need to first transform the categorical values into numerical values using `preprocessing.LabelEncoder()`.

References: 
1. Decision tree from sklearn: https://scikit-learn.org/stable/modules/tree.html

2. LabelEncoder from sklearn: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

3. Using LabelEncoder to transform categorical variables: https://stackoverflow.com/questions/38108832/passing-categorical-data-to-sklearn-decision-tree

## References:

1.机器学习 - 周志华

2.Hands-on Machine Learning with Scikit-Learn, Keras & TensorFlow, 2nd Edition - Aurélien Géron

3.统计学习方法（第2版）- 李航

4.ID3 algorithm - Wikipedia

5.数据挖掘十大算法（一）：决策树算法 python和sklearn实现 - CSDN

https://blog.csdn.net/qq_36523839/article/details/81408326

6.Decision Trees from Scratch Using ID3 Python: Coding It Up !!

https://medium.com/@lope.ai/decision-trees-from-scratch-using-id3-python-coding-it-up-6b79e3458de4

7.python：从零散的字典组装成树状嵌套字典 - CSDN

https://blog.csdn.net/qq_17065591/article/details/107528137

8.Python嵌套字典的遍历 - CSDN

https://blog.csdn.net/Tw_light/article/details/104961524